## Spark 설정하기
1. SparkConf, SparkContext, pandas 임포트
2. SparkConf 객체 생성 및 설정
3. SparkContext 객체 생성

In [1]:
from pyspark import SparkConf, SparkContext
import pandas as pd

In [2]:
conf = SparkConf().setMaster('local').setAppName('key-value-rdd') 

# SparkConf객체 생성

SparkConf객체 생성
- SparkConf 객체 : Spark 실행할 때 옵션 정보들 저장하는 객체  
- SparkConf() : Constructor라는 클래스의 객체 생성 메서드. 클래스 사용 위해서는 항상 객체를 생성하는 Constructor method를 선언해줘야함.
- 객체가 만들어지고 메모리에 존재하기 위해서는 변수를 만들어 저장해야함.
- setMaster옵션: 기본적으로 설정해야함(Master노드를 결정하는 것) / 현재는 하나의 pc로 하기 때문에 local로 설정(현재 pc)  
- setAppName: Spark 어플리케이션의 이름 설정

In [3]:
sc = SparkContext(conf=conf)

- 생성한 SparkConf객체를 SpartContext객체 생성할 때 인자로 넣어줌.  
- conf라는 parameter는 필수로 넣어줘야하고 인자값으로 conf객체 넣어주기  
- SparkContext를 일반적으로 sc라고 함  

In [4]:
sc 
# sparkContext에 대한 주요 내용들 나옴
# Spark UI누르면 모니터링으로 넘어감. 
# shift누르고 가면 새 탭 열면서 실시간으로 보여줌(안누르면 jupyter화면 그대로 넘어감)

<SparkContext master=local appName=key-value-rdd>

## Key-Value RDD 연산 예제
1. Python LIst로부터 RDD 객체 생성: SparkContext.parallelize() 
    => 간단한 데이터를 즉석에서 정의해서 RDD로 만들 때 사용하는 방법임.
2. Single Value RDD에서 Key-Value RDD로 변환: RDD.map()
3. Key-Value RDD 값 세기: RDD.reduceByKey()

지난 시간엔 RDD 생성할 땐 textFile()로 csv 파일 읽어서 RDD생성했었음. 이번에는 python list로부터 생성

In [5]:
# 1. RDD 객체 생성
rdd = sc.parallelize([
    '2020-03-01', '2020-03-01', '2020-03-01',
    '2020-03-02',
    '2020-03-03', '2020-03-03'
])

In [7]:
rdd.collect() # collect(): RDD 원소를 출력하는 action함수 -> 개발 단계에서 사용

['2020-03-01',
 '2020-03-01',
 '2020-03-01',
 '2020-03-02',
 '2020-03-03',
 '2020-03-03']

In [9]:
# 2. Key-Value RDD로 변환
pairs = rdd.map(lambda x: (x, 1))

In [10]:
pairs.collect()

[('2020-03-01', 1),
 ('2020-03-01', 1),
 ('2020-03-01', 1),
 ('2020-03-02', 1),
 ('2020-03-03', 1),
 ('2020-03-03', 1)]

In [11]:
# 3. Key-Value RDD 값 세기
reduced = pairs.reduceByKey(lambda x, y : x + y)

In [12]:
reduced.collect()

[('2020-03-01', 3), ('2020-03-02', 1), ('2020-03-03', 2)]

## Restaurant 예제
:식당 유형 별 평균 리뷰 수를 계산  
-> 식당 유형이 반드시 key로 있어야함
-> 평균 리뷰수에는 총 리뷰수가 필요. 전체를 각 개수로 나눠야하니까

1. csv 파일로부터 RDD 생성: SparkContext.textFile()
2. RDD 헤더 부분 변수에 저장: RDD.first()
3. 헤더만 제거된 RDD로 변환: RDD.filter(lambda...)
4. category와 reviews 값을 추출하는 함수 정의
5. category, review로 구성된 RDD로 변환: RDD.map()
6. Key: category, Value: (reviews, 1)로 구성된 Key-Value RDD로 변환: RDD.mapValues()
7. Key를 기준으로 reduction: RDD.reduceByKey()
8. Key별 평균 reviews 계산: RDD.mapValues()

In [13]:
# 1. RDD생성
filename = "restaurant_reviews.csv"
lines = sc.textFile('./data/' + filename) # ./ : 상대경로, 현재 실행중인 파일이 위치한 디렉토리

In [14]:
'./data/' + filename

'./data/restaurant_reviews.csv'

In [15]:
lines.collect() # 주의! RDD 데이터가 클 경우 자제 => 실행시간이 굉장히 오래 걸림

['id,item,cateogry,reviews',
 '0,짜장면,중식,125',
 '1,짬뽕,중식,235',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [21]:
# 2. header 구한 다음 뗴주기
header = lines.first() # 첫 번째 원소 => header 떼주기(데이터 분석할 때 header가 필요한게 아니니까)

In [22]:
header

'id,item,cateogry,reviews'

In [23]:
# header 제외한 나머지 줄들을 뽑아오는 것이 filter연산
filtered_lines = lines.filter(lambda x: x != header)

In [24]:
filtered_lines.collect()

['0,짜장면,중식,125',
 '1,짬뽕,중식,235',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [25]:
# 4. category, reviewr 값을 추출하는 함수 정의

def parse(row):
    # '0',짜장면,중식,125' -> 4개의 값이 아니라 하나의 문자열임. 각각 쪼개서 값으로 만들어야함.
    fields = row.split(',') # 값 쪼갬
    category = fields[2]
    reviews = int(fields[3]) # '125' -> 125
    return (category, reviews) # 묶어서 pair형태로 출력
    
# 전체 한 줄에서 식당 유형과 리뷰수만 따로 빼는 함수 만들기

In [27]:
# 5, category, review로 구성된 RDD로 변환: RDD.map()
category_reviews = filtered_lines.map(parse) # parse함수 정의했으니까 lambda 말고 parse

In [28]:
category_reviews.collect() # 잘 가져왔는지 확인

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [33]:
# 6. Key: category, Value: (reviews, 1)로 구성된 Key-Value RDD로 변환: RDD.mapValues()
# 주의! 그냥 map으로 하면 single value가 Key가 돼버림 (('중식',125), 1)
category_reviews_count = category_reviews.mapValues(lambda x: (x, 1))

In [34]:
category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [35]:
# 7. Key를 기준으로 reduction: RDD.reduceByKey()
reduced = category_reviews_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [36]:
reduced.collect() # 중식은 2개 식당 있고 360개 리뷰

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [39]:
# 8. Key별 평균 reviews 계산: RDD.mapValues()
averages = reduced.mapValues(lambda x: x[0] / x[1])

In [40]:
averages.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

## pyspark.RDD.collect(): RDD의 내용을 출력

In [41]:
"""
    Another method without 'parse' function -> category와 reviews만 떼주는 방법
"""

paired = filtered_lines.map(lambda x: (x.split(',')[2], (int(x.split(',')[3]), 1)))